In [ ]:
import gurobipy
from gurobipy import Model, GRB, quicksum


def build_model(
    S,
    G,
    d_g,  # {g: demand this month}
    c,  # {s: capacity}
    e,  # {(s,g): 0/1 eligibility}
    dist,  # {s: distance from hotspot}
    type="integer",  # 'integer',  'linear'. 'mixed'
    time_limit=None,
    mip_gap=None,
):
    # Validation
    if type not in {"integer", "linear", "mixed"}:
        raise ValueError("type must be one of {'integer','linear','mixed'}")

    x_vtype = GRB.INTEGER if type in {"integer", "mixed"} else GRB.CONTINUOUS
    z_vtype = GRB.INTEGER if type == "integer" else GRB.CONTINUOUS

    m = Model("Shelter_One_Hotspot")

    # Decision variables
    x = {
        (s, g): m.addVar(vtype=x_vtype, lb=0.0, name=f"x[{s},{g}]")
        for s in S
        for g in G
    }
    z = {g: m.addVar(vtype=z_vtype, lb=0.0, name=f"z[{g}]") for g in G}
    m.update()

    # Demand balance per gender
    for g in G:
        m.addConstr(
            quicksum(x[(s, g)] for s in S) + z[g] == d_g[g], name=f"demand[{g}]"
        )

    # Shelter capacity
    for s in S:
        m.addConstr(quicksum(x[(s, g)] for g in G) <= c[s], name=f"cap[{s}]")

    # Eligibility
    for s in S:
        for g in G:
            m.addConstr(x[(s, g)] <= e[(s, g)] * d_g[g], name=f"elig[{s},{g}]")

    # Pptional solver params
    if time_limit is not None:
        m.Params.TimeLimit = time_limit
    if mip_gap is not None:
        m.Params.MIPGap = mip_gap

    # STEP 1: Minimize unsheltered
    m.setObjective(quicksum(z[g] for g in G), GRB.MINIMIZE)
    m.optimize()
    if m.status not in (GRB.OPTIMAL, GRB.TIME_LIMIT):
        raise RuntimeError(f"Step 1 failed. Status={m.status}")

    Z_star = sum(z[g].X for g in G)

    # STEP 2: Fix unsheltered sum to Z* and minimize total distance
    m.addConstr(quicksum(z[g] for g in G) == Z_star, name="fix_unsheltered")
    m.setObjective(quicksum(dist[s] * x[(s, g)] for s in S for g in G), GRB.MINIMIZE)
    m.optimize()
    if m.status not in (GRB.OPTIMAL, GRB.TIME_LIMIT):
        raise RuntimeError(f"Step 2 failed. Status={m.status}")

    return m, {"x": x, "z": z}, {"Z_star": Z_star, "obj_distance": m.ObjVal}

In [ ]:
def run_12_months(
    monthly_demands,  # list of 12 items: either numbers (totals) or dicts {"men": x, "women": y}
    S,
    G,
    c,  # {s: capacity}
    e,  # {(s,g): 0/1}
    dist,  # {s: distance}
    gender_split=None,  # {g: share}, only used if monthly_demands contains totals
    type="integer",
    mip_gap=0.005,
    time_limit=None,
    print_utilization=True,
):
    results = []

    # need gender split only if monthly_demands has totals
    if any(not isinstance(dm, dict) for dm in monthly_demands):
        if not gender_split or abs(sum(gender_split.values()) - 1.0) > 1e-8:
            raise ValueError(
                "When monthly_demands contains totals, provide gender_split summing to 1.0"
            )

    for month_idx, dm in enumerate(monthly_demands, start=1):
        # Build d_g for this month
        if isinstance(dm, dict):
            d_g = {g: float(dm.get(g, 0.0)) for g in G}
        else:
            total = float(dm)
            d_g = {g: total * float(gender_split[g]) for g in G}

        # Solve
        m, vars_pack, info = build_model(
            S, G, d_g, c, e, dist, type=type, mip_gap=mip_gap, time_limit=time_limit
        )
        x, z = vars_pack["x"], vars_pack["z"]

        # Assignments and unsheltered
        assign = {(s, g): x[(s, g)].X for s in S for g in G if x[(s, g)].X > 1e-6}
        unshel = {g: z[g].X for g in G if z[g].X > 1e-6}

        # Totals assigned (overall and by gender)
        total_assigned = sum(x[(s, g)].X for s in S for g in G)
        assigned_by_gender = {g: sum(x[(s, g)].X for s in S) for g in G}

        # Average distance overall (km/person), guard divide-by-zero
        avg_distance = (
            info["obj_distance"] / total_assigned if total_assigned > 0 else 0.0
        )

        # Average distance by gender
        avg_distance_by_gender = {}
        for g in G:
            dist_sum_g = sum(dist[s] * x[(s, g)].X for s in S)
            cnt_g = assigned_by_gender[g]
            avg_distance_by_gender[g] = (dist_sum_g / cnt_g) if cnt_g > 0 else 0.0

        # Shelter utilization
        shelter_usage = {}
        for s in S:
            used = sum(x[(s, g)].X for g in G)
            cap = c[s]
            util_pct = (used / cap * 100.0) if cap > 0 else 0.0
            shelter_usage[s] = {
                "used": used,
                "capacity": cap,
                "utilization_pct": util_pct,
            }

        if print_utilization:
            print(
                f"\nMonth {month_idx:2d} | Z*={info['Z_star']:.0f} | "
                f"distance_total={info['obj_distance']:.1f} | "
                f"assigned={total_assigned:.0f} | "
                f"avg_distance={avg_distance:.3f} km/person"
            )
            print(
                "  Avg distance by gender: "
                + ", ".join(f"{g}={avg_distance_by_gender[g]:.3f}" for g in G)
            )
            print("Shelter utilization:")
            for s in S:
                su = shelter_usage[s]
                print(
                    f"  {s}: {su['used']:.0f} / {su['capacity']} ({su['utilization_pct']:.1f}%)"
                )

        results.append(
            {
                "month": month_idx,
                "Z_star": info["Z_star"],
                "obj_distance": info["obj_distance"],
                "assigned_total": total_assigned,
                "avg_distance": avg_distance,
                "avg_distance_by_gender": avg_distance_by_gender,
                "assignment": assign,
                "unsheltered": unshel,
                "shelter_usage": shelter_usage,
            }
        )

    return results


if __name__ == "__main__":
    G = ["men", "women"]

    S = [
        "973 Lansdowne Ave",
        "850 Bloor St W",
        "1651 Sheppard Ave W",
        "38 Bathurst St",
        "705 Progress Ave",
        "731 Runnymede Rd",
        "339 George St",
        "76 Church St",
        "674 Dundas St W",
        "616 Vaughan Rd",
        "349 George St",
        "386 Dundas St E",
        "512 Jarvis St",
        "67 Adelaide St E",
        "1059 College Street",
        "412 Queen St E",
        "35 Sydenham St",
        "702 Kennedy Rd",
        "14 Vaughan Rd",
        "26 Vaughan Rd",
        "962 Bloor St W",
        "126 Pape Ave",
        "60 Newcastle St",
        "70 Gerrard St E",
        "3410 Bayview Ave",
        "87 Pembroke St",
        "2808 Dundas St W",
        "107 Jarvis St",
        "135 Sherbourne St",
        "29A Leslie St",
        "2671 Islington Ave",
        "346 Spadina Ave.",
        "502 Spadina Ave",
        "80 Woodlawn Ave E",
        "348 Davenport Road",
    ]

    # capacities per shelter
    c = {
        "512 Jarvis St": 28,
        "348 Davenport Road": 28,
        "502 Spadina Ave": 65,
        "70 Gerrard St E": 44,
        "349 George St": 30,
        "339 George St": 122,
        "346 Spadina Ave.": 71,
        "87 Pembroke St": 30,
        "80 Woodlawn Ave E": 31,
        "674 Dundas St W": 88,
        "386 Dundas St E": 37,
        "135 Sherbourne St": 264,
        "67 Adelaide St E": 52,
        "107 Jarvis St": 78,
        "76 Church St": 53,
        "14 Vaughan Rd": 54,
        "35 Sydenham St": 5,
        "26 Vaughan Rd": 17,
        "412 Queen St E": 35,
        "850 Bloor St W": 36,
        "38 Bathurst St": 68,
        "962 Bloor St W": 16,
        "1059 College Street": 36,
        "126 Pape Ave": 24,
        "973 Lansdowne Ave": 42,
        "616 Vaughan Rd": 28,
        "29A Leslie St": 52,
        "2808 Dundas St W": 86,
        "731 Runnymede Rd": 62,
        "60 Newcastle St": 41,
        "1651 Sheppard Ave W": 18,
        "702 Kennedy Rd": 61,
        "3410 Bayview Ave": 30,
        "2671 Islington Ave": 46,
        "705 Progress Ave": 101,
    }

    # eligibility matrix
    e = {}
    # Men-only shelters
    for s in [
        "973 Lansdowne Ave",
        "850 Bloor St W",
        "1651 Sheppard Ave W",
        "38 Bathurst St",
        "705 Progress Ave",
        "731 Runnymede Rd",
        "339 George St",
        "76 Church St",
        "616 Vaughan Rd",
        "349 George St",
        "412 Queen St E",
        "35 Sydenham St",
        "14 Vaughan Rd",
        "26 Vaughan Rd",
        "107 Jarvis St",
        "135 Sherbourne St",
        "29A Leslie St",
        "2671 Islington Ave",
        "346 Spadina Ave.",
        "502 Spadina Ave",
    ]:
        e[(s, "men")] = 1
        e[(s, "women")] = 0

    # Women-only shelters
    for s in [
        "674 Dundas St W",
        "386 Dundas St E",
        "512 Jarvis St",
        "67 Adelaide St E",
        "1059 College Street",
        "702 Kennedy Rd",
        "962 Bloor St W",
        "126 Pape Ave",
        "60 Newcastle St",
        "70 Gerrard St E",
        "3410 Bayview Ave",
        "87 Pembroke St",
        "2808 Dundas St W",
        "80 Woodlawn Ave E",
        "348 Davenport Road",
    ]:
        e[(s, "men")] = 0
        e[(s, "women")] = 1

    # distance from the single hotspot (e.g., centroid) to each shelter
    dist = {
        "512 Jarvis St": 0.9778932317042026, 
        "348 Davenport Road": 1.1309174090050187,
        "502 Spadina Ave": 1.3086921105796272,
        "70 Gerrard St E": 1.3863911247008858,
        "349 George St": 1.6390483651607097,
        "339 George St": 1.670482508636295,
        "346 Spadina Ave.": 1.671288187435099,
        "87 Pembroke St": 1.8028318046617666,
        "80 Woodlawn Ave E": 1.8358389600864988,
        "674 Dundas St W": 2.0753002874685422,
        "386 Dundas St E": 2.1287228578113178,
        "135 Sherbourne St": 2.286022420867205,
        "67 Adelaide St E": 2.317295394322548,
        "107 Jarvis St": 2.3426292183615534,
        "76 Church St": 2.349838566145546,
        "14 Vaughan Rd": 2.6231083804920616,
        "35 Sydenham St": 2.6662125886876513,
        "26 Vaughan Rd": 2.674099501757083,
        "412 Queen St E": 2.6880854053791277,
        "850 Bloor St W": 2.7231762898874523,
        "38 Bathurst St": 3.060842441302751,
        "962 Bloor St W": 3.116770102477562,
        "1059 College Street": 3.6218320363280427,
        "126 Pape Ave": 4.3234509238481555,
        "973 Lansdowne Ave": 4.415402730126107,
        "616 Vaughan Rd": 4.713309356578003,
        "29A Leslie St": 5.092697798696732,
        "2808 Dundas St W": 5.797254555332866,
        "731 Runnymede Rd": 7.399278048600905,
        "60 Newcastle St": 10.218275618604487,
        "1651 Sheppard Ave W": 11.827495766539702,
        "702 Kennedy Rd": 12.059850994970251,
        "3410 Bayview Ave": 14.446273738400485,
        "2671 Islington Ave": 16.106719124859364,
        "705 Progress Ave": 16.905556379975213,
    }

    # Option A: totals by month (12 values), split 70/30
    monthly_by_gender = [
        {"men": 4979, "women": 2344},  # 2024-01-01
        {"men": 5083, "women": 2408},  # 2024-02-01
        {"men": 4872, "women": 2306},  # 2024-03-01
        {"men": 4775, "women": 2297},  # 2024-04-01
        {"men": 4629, "women": 2277},  # 2024-05-01
        {"men": 4507, "women": 2250},  # 2024-06-01
        {"men": 4396, "women": 2251},  # 2024-07-01
        {"men": 4397, "women": 2251},  # 2024-08-01
        {"men": 4288, "women": 2184},  # 2024-09-01
        {"men": 4259, "women": 2200},  # 2024-10-01
        {"men": 4380, "women": 2206},  # 2024-11-01
        {"men": 4925, "women": 2334},  # 2024-12-01
    ]

    results = run_12_months(
        monthly_demands=monthly_by_gender,  # list of 12 dicts: {"men": ..., "women": ...}
        S=S,
        G=G,
        c=c,
        e=e,
        dist=dist,
        type="mixed",
        mip_gap=0.005,
    )

    for r in results:
        print(
            f"Month {r['month']:2d} | Z*={r['Z_star']:.0f} | distance={r['obj_distance']:.1f}"
        )

Set parameter MIPGap to value 0.005
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  0.005

Optimize a model with 107 rows, 72 columns and 212 nonzeros
Model fingerprint: 0x82405d1c
Variable types: 2 continuous, 70 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+03]
Found heuristic solution: objective 7323.0000000
Presolve removed 107 rows and 72 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 5444 7323 

Optimal solution found (tolerance 5.00e-03)
Best objective 5.444000000000e+03, best bound 5.444000000000e+03, gap 0.0000%
Gurobi Optimize